In [1]:
import numpy as np
import pandas as pd
import math
import SALib
from SALib.sample import latin
import plotly.graph_objects as go

In [2]:
# Define fontsizes
lab_font_size = 20
tick_font_size = 18
anno_font_size = 16
fig_path = '../../output/paper3/'

In [3]:
# Define the function
# Parameters x are defined on [-1, 1]
def diff_power(x1, x2):
    """
    Parameters:
    x1, x2: np.ndarray, matrix of parameter values for the Gaussian peak integrand family (GPI).
    
    Return:
    y: np.ndarray, the output of SDP function.
    """
    y = math.exp(-25 * (x1 - 0.5)**2 - 4*(x2 - 0.5) ** 2 - 5*(x1-0.5)*(x2-0.5))
    return y
# vetorize diff_power fucntion 
v_func = np.vectorize(diff_power)

## Create the contour plot

In [4]:
# Create meshgrid points
x = np.array(np.meshgrid(np.linspace(0, 1, 1000),
                   np.linspace(0, 1, 1000)))

# Fix parameters at certain points and calculate the related  results
# Fix x2 
fix_1 = 0.5
fix_2 = 0.7
y_thres = 0.8
# Calculate the unconditioned results
z = v_func(x[0], x[1])

In [10]:
fig = go.Figure(data =
    go.Contour(
        z = z,
        x = x[0][0],
        y = x[1][:,0],
        colorbar = dict(title='Values', # title here
            titleside='top',
            titlefont=dict(
            size=lab_font_size),
            tickfont = dict(size = tick_font_size)
            ),
        colorscale='sunset',
    contours=dict(
            coloring ='heatmap',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = 12,
                color = 'white'),
            start=0,
            end=1,
            size=0.1)
            ))

fig.add_trace(go.Scatter(x=[0.001, 0.001, 0.999, 0.999, 0.001], y=[0, 1, 1, 0, 0], 
                         fill=None, line_color='royalblue',
                        line_width=2, mode='lines', showlegend=False))

# Add annotation and arrows describing box
fig.add_annotation(x=0.55, y=0.74,
            text="The bounding box of feasible set",
            showarrow=True,
            arrowhead=1,
            font = {"size": anno_font_size}
            )
# Add markers indicating optima
fig.add_trace(go.Scatter(x=[0.5, 0.48], y=[fix_1, fix_2], \
                         mode='markers', marker={'symbol':2,'size':8, 'color':['red', 'green']}, showlegend=False))

# Add annotations to contour plot
fig.add_hline(y=fix_1, line_width=1.5, line_dash="dash", line_color="red", 
              annotation_text = r'$f(x_1, x_2=%.1f)$'%fix_1, annotation_font_size = anno_font_size)
fig.add_hline(y=fix_2, line_width=1.5, line_dash="dash", line_color="green", 
              annotation_text = r'$f(x_1, x_2=%.1f)$'%fix_2, annotation_font_size = anno_font_size)
fig.update_xaxes(title_text = r'$x_1$', title_font = {"size": lab_font_size}, tickfont=dict(size=tick_font_size))
fig.update_yaxes(title_text = r'$x_2$',title_font = {"size": lab_font_size},  tickfont=dict(size=tick_font_size))
fig.write_image(f'{fig_path}contour_1114.png')

## Create boxplot

In [7]:
# random sampling to over parameter space and then fix variable x2
problem = {'num_vars': 2,
          'names': ['x1', 'x2'],
          'bounds': [[0, 1], 
                    [0, 1]]
          }
sample_size = 1000
param_values_full = latin.sample(problem, sample_size)
x1_full = param_values_full[:, 0]
x2_full = param_values_full[:, 1]
z_uncon = v_func(x1_full, x2_full)

# Calculate the conditional values with x2 fixed.
group_labels = [r'$f(x_1, x_2)$', r'$f(x_1, x_2=%.1f)$'%(fix_1), r'$f(x_1, x_2=%.1f)$'%(fix_2)]
z_c1 = v_func(x1_full, fix_1)
z_c2 = v_func(x1_full, fix_2)

In [8]:
# Form data into a dataframe before using seaborn
hist_data = [z_uncon, z_c1, z_c2]
df_plot = pd.DataFrame(columns=['x', 'H values', 'Type'], index = np.arange(sample_size * 6))
for xx, yy, indd in zip(group_labels, hist_data, [999, 1999, 2999]):
    df_plot.loc[(indd-999):indd, 'x'] = xx
    df_plot.loc[(indd-999):indd, 'H values'] = yy
df_plot.loc[:, 'Type'] = 'All'

# add data satisfying the threshold: H>=0.8
ind_start = 3000
for xx, yy in zip(group_labels, hist_data):
    ind =  np.where(yy >= y_thres)[0]
    df_plot.loc[ind_start: (ind_start + ind.shape[0]-1 ), 'x'] = xx
    df_plot.loc[ind_start: (ind_start + ind.shape[0]-1), 'H values'] = yy[ind]
    df_plot.loc[ind_start: (ind_start + ind.shape[0]-1), 'Type'] = 'H>=0.8'
    ind_start  = ind_start + ind.shape[0]    

In [10]:
hist_data = [z_uncon, z_c1, z_c2]
q_1 = np.quantile(hist_data, 0.25, axis=1)
q_3 = np.quantile(hist_data, 0.75, axis=1)
q_m = np.quantile(hist_data, 0.50, axis=1)
lower_bounds = np.quantile(hist_data, 0.025, axis=1)
upper_bounds = np.quantile(hist_data, 0.975, axis=1)

# Dataframe of selected data
z_uncon_fil = z_uncon[np.where(z_uncon >= y_thres)[0]]
z_c1_fil = z_c1[np.where(z_c1 >= y_thres)[0]]
z_c2_fil = z_c2[np.where(z_c2 >= y_thres)[0]]

x1_fil = x1_full[np.where(z_uncon >= y_thres)[0]]
x1_c1 = x1_full[np.where(z_c1 >= y_thres)[0]]
x1_c2 = x1_full[np.where(z_c2 >= y_thres)[0]]

# Quantiles for selected data points
hist_data_fil = np.array([z_uncon_fil, z_c1_fil, z_c2_fil])
q_1_fil = [np.quantile(hist_data_fil[ii], 0.25) for ii in range(3)]
q_3_fil = [np.quantile(hist_data_fil[ii], 0.75) for ii in range(3)]
q_m_fil = [np.quantile(hist_data_fil[ii], 0.50) for ii in range(3)]
lower_bounds_fil = [np.quantile(hist_data_fil[ii], 0.025) for ii in range(3)]
upper_bounds_fil = [np.quantile(hist_data_fil[ii], 0.975) for ii in range(3)]

C:\Users\hp\AppData\Local\Temp\ipykernel_54016\2426285774.py:18: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [45]:
fig3 = go.Figure()
# Plot boxs for all points
fig3.add_trace(go.Box(y = np.array(hist_data), width = 0.2, 
                     q1=q_1, median=q_m,
                  q3=q_3, lowerfence = lower_bounds,
                  upperfence = upper_bounds, boxpoints='outliers', name='All'))

# Plot boxs for selected points
fig3.add_trace(go.Box(y = np.array(hist_data_fil), width = 0.2, 
                     q1=q_1_fil, median=q_m_fil,
                  q3=q_3_fil, lowerfence = lower_bounds_fil,
                  upperfence = upper_bounds_fil, boxpoints='outliers', x0=0.2, 
                      name='Feasible'))

fig3.update_yaxes(title_text = r'$H\ values$', title_font = {"size": lab_font_size}, tickfont={"size" : tick_font_size})
fig3.update_xaxes(ticktext = group_labels, tickvals=[0, 1, 2],\
                  tickfont={"size" : tick_font_size}, title_font = {"size": lab_font_size})
fig3.update_layout(legend=dict(font={'size':tick_font_size},yanchor = "top",
                              y = 0.25,
                              xanchor = "left",
                              x = 0.15))
fig3.write_image(f'{fig_path}boxplot.png', format = 'png', scale=1)

In [17]:
# fig4 = go.Figure()
# fig4.add_trace(go.Box(y = np.array(hist_data_fil), width = 0.2, 
#                      q1=q_1_fil, median=q_m_fil,
#                   q3=q_3_fil, lowerfence = lower_bounds_fil,
#                   upperfence = upper_bounds_fil, boxpoints='outliers', x0=0.2, 
#                       name='Selected', fillcolor='orange', line_color='orange'))

# fig4.update_yaxes(title_text = r'$H\ values$', title_font = {"size": lab_font_size}, tickfont={"size" : tick_font_size})
# fig3.update_xaxes(ticktext = group_labels, tickvals=[0, 1, 2],\
#                   tickfont={"size" : tick_font_size}, title_font = {"size": lab_font_size})
# fig4.update_layout(legend=dict(font={'size':tick_font_size}))
# fig4.write_image(f'{fig_path}boxplot-select.png', format = 'png', scale=1)

## Create scatter plot

In [33]:
# Scatter plot of x2 fixed at fix_1
fig5 = go.Figure()
k = 0
for ydata, lab in zip(hist_data, group_labels):
    if k != 1:  
        fig5.add_trace(go.Scatter(x = x1_full, y = ydata, mode='markers', name=lab, 
                                  marker = {'size': 6, 'opacity' : 0.7}))
    else:
        fig5.add_trace(go.Scatter(x = x1_full, y = ydata, mode='markers', name=lab, 
                                  marker = {'size': 6, 'opacity' : 0.2}))
    k += 1
fig5.update_yaxes(title_text = r'$H\ values$', title_font = {"size": lab_font_size}, tickfont = {"size" : tick_font_size})
fig5.update_xaxes(title_text = r'$x_1$', title_font = {"size": lab_font_size}, tickfont = {"size" : tick_font_size})
fig5.update_layout(legend=dict(yanchor = "top",
                              y = 0.95,
                              xanchor = "left",
                              x = 0.65, 
                              font = {"size":14}))

fig5.add_vline(x=0.5, line_width=1.5, line_dash="dash", line_color="red", 
              annotation_text = '')
fig5.add_vline(x=0.475, line_width=1.5, line_dash="dash", line_color="green", 
              annotation_text = '')
fig5.write_image(f'{fig_path}scatters-full.png', format = 'png', scale=2)

In [37]:
# Scatter plot of x2 fixed at fix_1
x1_data = [x1_fil, x1_c1, x1_c2]
fig6 = go.Figure()
k = 0
for xx1, ydata, lab in zip(x1_data, hist_data_fil, group_labels):
    if k != 1:  
        fig6.add_trace(go.Scatter(x = xx1, y = ydata, mode='markers', name=lab, 
                                  marker = {'size': 6, 'opacity' : 0.7}))
    else:
        fig6.add_trace(go.Scatter(x = xx1, y = ydata, mode='markers', name=lab, 
                                  marker = {'size': 6, 'opacity' : 0.2}))
    k += 1
fig6.update_yaxes(title_text = r'$H\ values$', title_font = {"size": lab_font_size}, tickfont = {"size" : tick_font_size})
fig6.update_xaxes(title_text = r'$x_1$', title_font = {"size": lab_font_size}, tickfont = {"size" : tick_font_size})
# fig6.update_layout(legend=dict(yanchor = "top",
#                               y = 0.95,
#                               xanchor = "left",
#                               x = 0.65, 
#                               font = {"size":14}))
fig6.update_layout(showlegend=False)

fig6.add_vline(x=0.5, line_width=1.5, line_dash="dash", line_color="red", 
              annotation_text = '')
fig6.add_vline(x=0.475, line_width=1.5, line_dash="dash", line_color="green", 
              annotation_text = '')
fig6.write_image(f'{fig_path}scatters-select.png', format = 'png', scale=2)